In [6]:
search_for = 101
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.01268315315246582
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 105101005
type: [1, 1, 1, 1, 101] 101
cases of this type: 104060401
10000 12.694460111790994
20000 11.42486665245662
30000 12.694838778086469
40000 53.1198156711936
50000 55.099211681378875
60000 58.14086166263481
70000 64.1451042296638
80000 67.5368316743612
90000 65.99436669307487
100000 65.73738283720107
110000 66.06718462256133
120000 65.79932616716127
130000 65.23592806504995
140000 65.24513137056526
150000 65.7325726440636
160000 64.15458113179515
170000 64.17762910903613
180000 63.82764679928183
190000 63.30270413457324
200000 64.31258221307061
210000 64.76258974857558
220000 64.98934384233326
230000 63.84523037809067
240000 63.46600875672648
250000 68.95089987826815
260000 73.73237285150498
270000 72.81648661437107
280000 66.32109327127259
290000 63.85006606972781
300000 64.24090493283865
310000 64.41298479816795
320000 64

3120000 17.388150411579808
3130000 17.46881706789424
3140000 17.48761500629988
3150000 17.326097467809454
3160000 60.58909475675304
3170000 64.07914261995727
3180000 57.500242468258776
3190000 57.738562337957916
3200000 59.894239498374496
3210000 60.61762554882779
3220000 62.123549263647796
3230000 60.55263021464953
3240000 61.534064018554
3250000 61.40739570424835
3260000 59.42376722463863
3270000 58.177272804221175
3280000 58.94996822141797
3290000 59.978057503617244
3300000 60.86244935565501
3310000 60.68041296517293
3320000 60.394305934185056
3330000 62.251092644456264
3340000 62.20717274324775
3350000 61.0275260427568
3360000 61.81435850014849
3370000 59.86489013101905
3380000 59.754989936990015
3390000 60.74502625575834
3400000 58.75125134130557
3410000 59.231700373942324
3420000 58.42593522249126
3430000 57.44333044760008
3440000 56.263218934429055
3450000 57.23159099752493
3460000 57.00091738060176
3470000 57.55644836515228
3480000 58.091219650918376
3490000 57.934757883171734


6250000 16.973096029904486
6260000 17.253679008850355
6270000 16.964874707139067
6280000 51.17233386155257
6290000 65.28811625335584
6300000 58.819587047034105
6310000 58.48234235480934
6320000 59.16971833296659
6330000 58.717921932230134
6340000 58.7268133091256
6350000 56.525036711455954
6360000 54.34365249312662
6370000 54.89818750339488
6380000 55.269865136885386
6390000 56.219558265977106
6400000 55.98573566567222
6410000 58.827269403979344
6420000 60.159860661682686
6430000 58.70555606592833
6440000 58.39402478092469
6450000 58.22439551010701
6460000 59.54594794436614
6470000 62.52815654501064
6480000 60.68663477168627
6490000 62.23589140812278
6500000 56.66663992714146
6510000 54.535712178468806
6520000 55.05767068721027
6530000 55.40405730416759
6540000 54.708160722514535
6550000 55.27197326349437
6560000 57.11111222413789
6570000 59.730778100136284
6580000 60.07582461769103
6590000 57.11566324118473
6600000 57.15425947261392
6610000 56.317867005652786
6620000 56.56347174275372

9380000 16.335715559798743
9390000 16.335767315491104
9400000 42.303203207190336
9410000 63.69016370912535
9420000 57.30326330398616
9430000 55.62764722718139
9440000 56.26668427472181
9450000 56.38464355071902
9460000 55.77912224661261
9470000 56.68427243576
9480000 58.5952380233681
9490000 58.83457082503616
9500000 56.911373144237196
9510000 55.65041065355259
9520000 56.00093508028905
9530000 56.35397177576369
9540000 54.47869064844346
9550000 53.69678139670034
9560000 52.132264994544784
9570000 56.232442749924026
9580000 56.110879167097806
9590000 58.28746972629212
9600000 61.79203637959864
9610000 55.90164652307024
9620000 55.69685264183253
9630000 56.61077608016528
9640000 52.91858229112923
9650000 52.551163720370525
9660000 52.707702814612226
9670000 53.008134734402006
9680000 52.35072225354089
9690000 53.006046909384395
9700000 55.89303422528227
9710000 54.12872808816235
9720000 50.93862756996307
9730000 50.229196166175704
9740000 48.650144193768696
9750000 52.346644477862455
97

12420000 54.67880498621031
12430000 56.272756234178644
12440000 54.66700256264687
12450000 55.73628139630228
12460000 61.16613165811241
12470000 41.23448125793722
12480000 15.930159971689715
12490000 16.03704510830303
12500000 16.175582301214668
12510000 15.76807218394518
12520000 32.25638067596912
12530000 60.648622525888214
12540000 52.440630159328386
12550000 50.68790233788908
12560000 55.81362228081445
12570000 57.38518598045621
12580000 56.62952860418946
12590000 55.353958774223024
12600000 55.25457921189623
12610000 55.407897191134296
12620000 54.92133773657299
12630000 55.54343601585429
12640000 55.81318188028276
12650000 54.175398563947375
12660000 53.22811794073873
12670000 52.667159046586164
12680000 56.15322088840991
12690000 54.385521202935514
12700000 57.56958707687576
12710000 52.84051728248738
12720000 51.82020651995185
12730000 51.177862461814286
12740000 49.70671295903266
12750000 48.11028353052208
12760000 48.65052859270513
12770000 48.62087342338764
12780000 48.82031

...

93370000 7.316997351875006
93380000 6.614029303568039
93390000 6.704142290888361
93400000 7.1182222980422765
93410000 6.6686684068724515
93420000 6.611683130647307
93430000 6.875386455346743
93440000 7.730362143980099
93450000 7.722759894973338
93460000 7.941845181369483
93470000 7.44702603018562
93480000 7.1028881098560825
93490000 7.1369799492832025
93500000 7.084269520990385
93510000 7.032362102843888
93520000 6.847472176677882
93530000 7.0738312191174755
93540000 6.953232079511021
93550000 7.2246973496666556
93560000 7.572500990941757
93570000 7.523689120636449
93580000 7.359721440279682
93590000 6.9939709161722
93600000 6.749000175985892
93610000 7.309958948435287
93620000 8.005384717966615
93630000 5.12824917207443
93640000 2.145293564065695
93650000 2.0805274580451183
93660000 2.0717194202187987
93670000 2.000555617465874
93680000 4.907264081498285
93690000 8.05132940422111
93700000 7.227743626165987
93710000 6.859291574426128
93720000 7.1358297693341655
93730000 7.0230441273322

96400000 4.676077119145393
96410000 5.117611221600241
96420000 5.424023465107845
96430000 5.078927794499993
96440000 5.444765885446105
96450000 4.975412564013567
96460000 5.13377602834185
96470000 5.589367230589953
96480000 4.932577939741545
96490000 4.967363654364546
96500000 5.603319502680699
96510000 5.175816983985305
96520000 5.335953786355556
96530000 5.120636118232442
96540000 5.068672983402974
96550000 5.570635949377715
96560000 5.1090905222470235
96570000 5.359335151752399
96580000 5.549304095422229
96590000 4.964635787788563
96600000 5.188606413764557
96610000 5.259323530799151
96620000 4.880641266168389
96630000 5.312993136639429
96640000 5.0998435029359666
96650000 5.005707104353938
96660000 5.525221571506196
96670000 4.996668974550565
96680000 5.028535329911941
96690000 5.596755435186657
96700000 5.127037080370188
96710000 5.588046854009562
96720000 5.028704996360806
96730000 4.994870129284759
96740000 6.039147208562395
96750000 4.221033316655855
96760000 1.4450137090911468

99410000 0.4996305105653736
99420000 0.49362626109023894
99430000 0.4936324461748202
99440000 0.4898095898098747
99450000 0.5630102542454335
99460000 4.15279506561021
99470000 3.7578474993102087
99480000 3.019348255098429
99490000 3.183294046330154
99500000 3.697083721803195
99510000 3.4604986105251316
99520000 3.691622372700373
99530000 3.52431392787589
99540000 3.3329570167517995
99550000 3.6961190528440473
99560000 3.298366035269267
99570000 3.1483923285689945
99580000 3.437104537161887
99590000 3.1571221522281565
99600000 3.1295890469375913
99610000 3.4839590330386163
99620000 3.1747945271029407
99630000 3.5374536853042575
99640000 3.291733188414276
99650000 3.437092035816014
99660000 3.714212571157846
99670000 3.503281010713875
99680000 3.491573949608107
99690000 3.439566028370791
99700000 3.3254731781968476
99710000 3.760412985232506
99720000 3.4600749157481063
99730000 3.3137882036385933
99740000 3.5341330917389526
99750000 3.1859255570178893
99760000 3.2615634219145777
99770000

102300000 1.6914216523227428
102310000 1.719177759146889
102320000 1.7105172723500597
102330000 1.679833825505177
102340000 1.6446629101826749
102350000 1.688895513472789
102360000 1.6410701728116142
102370000 1.6345339122742415
102380000 1.6748805549557673
102390000 1.6496001948993075
102400000 1.638913504189352
102410000 1.6744006231847737
102420000 1.7185617562358246
102430000 1.6605666089956959
102440000 1.6513766277041035
102450000 1.632143736171325
102460000 1.6562582715529202
102470000 1.7077071767625545
102480000 1.8151177783587242
102490000 2.115151181460023
102500000 0.23659986720558668
102510000 0.23324970260587002
102520000 0.23527318386375906
102530000 0.2315226830876867
102540000 0.23213796135058004
102550000 2.206364510600567
102560000 1.8843412628176146
102570000 1.5978690806968345
102580000 1.6243149028990667
102590000 1.5726526017899314
102600000 1.5594618932967717
102610000 1.6001609346030157
102620000 1.5693739120137693
102630000 1.5783124928625756
102640000 1.57988

105060000 0.002832741974459754
105070000 0.002140002464354038
105080000 0.001454251273473104
105090000 0.0007650911772251128
best so far: 0
type: [1, 1, 101, 1, 1] 101
cases of this type: 10201
105100000 7.26195185383161e-05
best so far: 0
type: [1, 101, 1, 1, 1] 101
cases of this type: 101
best so far: 0
type: [101, 1, 1, 1, 1] 101
cases of this type: 1
best so far: 0
all cases: 105101005
best: 0
